In [1]:
import os
import PIL.Image as Image
import torch.cuda
import torchvision.transforms as transforms
import numpy as np

from LookGenerator.datasets.utils import prepare_images_for_encoder, to_array_from_decoder, load_image, convert_channel, show_array_as_image, save_array_as_image
from LookGenerator.networks.encoder_decoder import EncoderDecoder
from LookGenerator.networks.utils import load_model

# Загрузка изображений

In [2]:
root = r"C:\Users\DenisovDmitrii\Desktop\forEncoder\val"
file_name = r"00006_00"

human_image = load_image(root, r"imageWithNoCloth", file_name, ".png")
clothes = load_image(root, r"cloth", file_name, r".jpg")

pose_points = []
points_list = os.listdir(os.path.join(
    root,
    r"posePoints",
    file_name
))
print(len(points_list))
for pose_point in points_list:
    pose_point_image = convert_channel(load_image(root, os.path.join(r"posePoints", file_name), pose_point, ""))
    pose_points.append(pose_point_image)

# if model dataset has pose_points=False param:
pose_points = []

17


# Определение трансформаций

In [3]:
input_rgb_transform = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.25, 0.25, 0.25]
    )
])

input_bin_transform = transforms.Compose([
    transforms.Resize((256, 192)),
    transforms.Normalize(
        mean=[0.5],
        std=[0.25]
    )
])

# Загрузка весов модели

In [ ]:
model_input = prepare_images_for_encoder(human_image, pose_points, clothes, input_rgb_transform, input_bin_transform).float()

In [5]:
weights_dir = r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\encoder\weights\session12\epoch_19.pt"

In [6]:
model = EncoderDecoder(in_channels=6, out_channels=3)
model = load_model(model, weights_dir)

In [7]:
device = 'cpu' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
model_input = model_input.to(device)

# Прогон модели

In [8]:
model_output = model(model_input)

# Отображение результата

In [9]:
model_output = to_array_from_decoder(model_output)

In [10]:
to_plt = model_output

In [11]:
c = np.array(to_plt[:,:,0])
to_plt[:,:,0] = to_plt[:,:,2]
to_plt[:,:,2] = c

In [ ]:
show_array_as_image(np.invert(np.uint8(255*to_plt)))
save_array_as_image(np.uint8(255*model_output), save_path=fr"C:\Users\DenisovDmitrii\Desktop\forEncoder\out\{file_name}.png")
